In [4]:
#in the terminal: pip install torch torchvision torchaudio 
#in the terminal: pip install pytorch-lightning 
#in the terminal: pip install pandas 
#in the terminal: pip install pytorch-lightning pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu121/torch_stable.html
#in the terminal: pip install -U scikit-learn

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy
import pandas as pd
from torchmetrics.functional import accuracy
from torch.optim.lr_scheduler import StepLR
import numpy as np
from torchvision import transforms, datasets
from torchvision.transforms import Compose
from torch.utils.data import ConcatDataset
import random
from torchvision.transforms import ToTensor
from sklearn.metrics import roc_auc_score, f1_score, matthews_corrcoef
import itertools
from sklearn.utils.class_weight import compute_class_weight

c:\Users\120224294\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# df = pd.read_csv('nirs with HIE outcome for CNN_10hours.csv', header = None)
# label = pd.read_csv('HIE outcome for CNN_10hours.csv', header = None)
# signals = df.values
# signals = signals[180:,1:]
# print(signals.shape)
# # mean = np.mean(signals)
# # std = np.std(signals)
# # signals = (signals - mean)/std
# label = label.values
# label = label[180:,1]
# label = label.reshape(-1,1)
# # label = torch.tensor(label)
# print(label.shape)
# dataset = list(zip(signals, label))


In [2]:
def making_test_val_ids(df, n):
    IDs = df[0].unique()
    random_ids = pd.Series(IDs).sample(n , replace= False)
    return random_ids

def making_train_test_val_signals(df, label, hie, n):
    a = making_test_val_ids(df, n)
    train_signal = df[~df[0].isin(a)]
    train_label = label[~label[0].isin(a)]
    test_IDs = a[:(n//2)]
    val_IDs = a[(n//2):]
    # test_IDs = [22,15,12,21]
    # val_IDs = [49,37,55,43]
    train_signal = train_signal.values
    train_signal = train_signal[:,1:]
    train_hie = hie[~hie[0].isin(a)]
    train_hie = train_hie.values
    train_hie = train_hie[:,1]
    train_label = train_label.values
    train_label = train_label[:,1]
    test_signal = df[df[0].isin(test_IDs)]
    test_signal = test_signal.values
    test_signal = test_signal[:,1:]
    test_label = label[label[0].isin(test_IDs)]
    test_label = test_label.values
    test_label = test_label[:,1]
    test_hie = hie[hie[0].isin(test_IDs)]
    test_hie = test_hie.values
    test_hie = test_hie[:,1]
    val_signal = df[df[0].isin(val_IDs)]
    val_signal = val_signal.values
    val_signal = val_signal[:,1:]
    val_label = label[label[0].isin(val_IDs)]
    val_label = val_label.values
    val_label = val_label[:,1]
    val_hie = hie[hie[0].isin(val_IDs)]
    val_hie = val_hie.values
    val_hie = val_hie[:,1]
    print(test_IDs)
    print(val_IDs)
    return train_signal, train_label, train_hie, test_signal, test_label, test_hie,  val_signal, val_label, val_hie

def separate_epochs(train_signal, epoch_length):
    num_epochs = 5890 // epoch_length
    separated_epochs = np.split(train_signal, num_epochs, axis=1)
    return separated_epochs

def shuffle_epochs(epochs):
    np.random.shuffle(epochs)
    return epochs

def reconstruct_signals(epochs):
    reconstructed_signals = np.concatenate(epochs, axis=1)
    return reconstructed_signals

def separate_pieces(signal, num_pieces):
    piece_length = 5890 // num_pieces
    pieces = np.split(signal, num_pieces, axis=1)
    return pieces



def generate_new_signals(dataset, num_signals, num_rep, epoch_length, num):
    filtered_dataset = [(train_signal, train_hie, train_label) for train_signal, train_hie,  train_label in dataset if train_label == num and train_hie == num]
    filtered_signals = np.array([train_signal for train_signal, _ , _ in filtered_dataset])
    filtered_signals = filtered_signals.reshape(-1, 1, 5890)
    combined_signals = []
    for _ in range(num_rep):
        selected_indices = np.random.choice(filtered_signals.shape[0], size=num_signals, replace=False)
        selected_signals = filtered_signals[selected_indices]

        combined_epochs = []
        for train_signal in selected_signals:
            epochs = separate_epochs(train_signal, epoch_length)
            combined_epochs += epochs

        shuffled_epochs = shuffle_epochs(combined_epochs)
        reconstructed_signals = reconstruct_signals(shuffled_epochs)
        combined_signals.append(reconstructed_signals)

    combined_signals = np.array(combined_signals)
    # combined_signals = combined_signals.reshape(num_rep, 1, 5890)
    combined_signals = combined_signals.reshape(num_rep, -1)[ :, :5890]
    return combined_signals

def generate_new_signals_from_one(dataset, num_pieces):
    new_signals = []
    for signal, hie, label in dataset:
        signal = signal.reshape(1, 5890)
        pieces = separate_pieces(signal, num_pieces)
        permutations = list(itertools.permutations(pieces))
        for perm in permutations:
            # shuffled_pieces = shuffle_epochs(perm)
            new_signal = np.concatenate(perm, axis=1)
            new_signals.append((new_signal, hie, label))
    return new_signals


def generate_new_rows_amin(row, num_new_rows, window_percentage):
    new_rows = []
    for _ in range(num_new_rows):
        row_length = len(row)
        window_size = int(row_length * window_percentage)

        # Randomly select start indices for the windows
        start_idx_orig = random.randint(1, row_length - window_size)
        start_idx_swap = random.randint(1, row_length - window_size)

        # Generate the new row by swapping the windows
        new_row = row[:]
        new_row[start_idx_orig:start_idx_orig + window_size], new_row[start_idx_swap:start_idx_swap + window_size] = \
            new_row[start_idx_swap:start_idx_swap + window_size], new_row[start_idx_orig:start_idx_orig + window_size]

        new_rows.append(new_row)
    return new_rows

# def sigmoid(x):
#     return 1 / (1 + torch.exp(-x))

In [8]:
# df = pd.read_csv('nirs with HIE outcome for CNN_10hours.csv', header = None)
# label = pd.read_csv('HIE outcome for CNN_10hours.csv', header = None)
# train_signal, train_label, test_signal, test_label, val_signal, val_label = making_train_test_val_signals(df, label, 10)
# val_label = val_label.reshape(-1,1)
# val_signal = val_signal.reshape(-1,1,5890)
# test_label = test_label.reshape(-1,1)
# test_signal = test_signal.reshape(-1,1,5890)
# dataset = list(zip(train_signal, train_label))
# val_dataset = list(zip(val_signal, val_label))
# test_dataset = list(zip(test_signal, test_label))

In [9]:
# df_dict = {}
# for i in range(10):
#     if i <4:
#         train_label = 1
#         filename = f'combined_signals_train_label_1_{i}.csv'
#     else:
#         train_label = 0
#         filename = f'combined_signals_train_label_0_{i-4}.csv'

#     df_combined_signals = generate_new_signals(dataset,150, 1000, 589)
#     df_combined_signals = pd.DataFrame(df_combined_signals)
#     # df_dict[filename] = filename
#     df_combined_signals.to_csv(filename, header=None, index=False)

In [10]:
# df_00 = pd.read_csv('combined_signals_train_label_0_0.csv', header = None)
# df_01 = pd.read_csv('combined_signals_train_label_0_1.csv', header = None)
# df_02 = pd.read_csv('combined_signals_train_label_0_2.csv', header = None)
# df_03 = pd.read_csv('combined_signals_train_label_0_3.csv', header = None)
# df_04 = pd.read_csv('combined_signals_train_label_0_4.csv', header = None)
# df_05 = pd.read_csv('combined_signals_train_label_0_5.csv', header = None)
# df_10 = pd.read_csv('combined_signals_train_label_1_0.csv', header = None)
# df_11 = pd.read_csv('combined_signals_train_label_1_1.csv', header = None)
# df_12 = pd.read_csv('combined_signals_train_label_1_2.csv', header = None)
# df_13 = pd.read_csv('combined_signals_train_label_1_3.csv', header = None)


In [11]:
# combined_signals = np.concatenate([train_signal, df_00, df_01, df_02, df_03,
#                                    df_04, df_05, df_10, df_11, df_12, df_13], axis=0)
# arr0 = np.zeros((6000,1))
# arr1 = np.ones((4000,1))
# train_label = train_label.reshape(-1,1)
# combined_label = np.concatenate([train_label, arr0, arr1], axis = 0)
# combined_label = combined_label.reshape(-1,1)
# combined_signals = combined_signals.reshape(-1,1,5890)
# combined_signals = torch.tensor(combined_signals)
# combined_label = torch.tensor(combined_label)
# dataset = list(zip(combined_signals, combined_label))

In [12]:
# set1 = set(tuple(signal.flatten()) for signal in signals)
# set2 = set(tuple(signal.flatten()) for signal in combined_signals)

# # Check if there are any same signals between the two groups
# common_signals = set1.intersection(set2)

# # Print the common signals, if any
# if common_signals:
#     print("Common signals found:")
#     for signal in common_signals:
#         print(signal)
# else:
#     print("No common signals found.")

In [13]:
# signal_set = set(tuple(signal.flatten()) for signal in combined_signals)

# # Check if there are any duplicate signals in the dataset
# if len(signal_set) < len(combined_signals):
#     print("Duplicate signals found.")
# else:
#     print("No duplicate signals found.")

In [14]:
# weights = torch.tensor([0.679]).cuda()
# weights = torch.tensor([0.22]).cuda()

In [15]:
# def sigmoid(x):
#     return 1 / (1 + torch.exp(-x))

# def add_noise(x):
#     noise = torch.rand_like(x)*0.1
#     return x + noise 

# def time_shift(x):
#     shift = np.random.randint(-10,10)
#     return torch.roll(x, shift, dims= 1)

# transform = transforms.Compose([
#     transforms.Lambda(lambda x: x.unsqueeze(0)), 
#     transforms.Lambda(add_noise),
#     transforms.Lambda(time_shift),
#     transforms.Lambda(lambda x: x.unsqueeze(0))])

# augmented_x = transform(signals)
#augmented_dataset = list(zip(augmented_x, label))
# combined_dataset_train = ConcatDataset([dataset[180:], augmented_x[180:]])

# combined_dataset_train = ConcatDataset([dataset[180:] , df_0 , df_1])


In [ ]:
class CNN(pl.LightningModule):
  def __init__(self):
    super().__init__()
    # self.layer1 = nn.Sequential(
    #   nn.Conv1d(1, 64, kernel_size= 10, stride=1, padding=2),
    #   nn.LeakyReLU(),
    #   nn.MaxPool1d(kernel_size=2, stride=6))
    self.layer1 = nn.Sequential(
      nn.Conv1d(1, 16, kernel_size= 11, stride=2, padding=0),
      nn.BatchNorm1d(16),
      nn.LeakyReLU())
    # self.layer2 = nn.Sequential(
    #   nn.Conv1d(64, 64, kernel_size= 10, stride=1, padding=2),
    #   nn.LeakyReLU(),
    #   nn.MaxPool1d(kernel_size=2, stride=6))
    self.layer2 = nn.Sequential(
      nn.Conv1d(16, 32, kernel_size= 7, stride=2, padding=0),
      nn.BatchNorm1d(32),
      nn.LeakyReLU())
    # self.layer3 = nn.Sequential(
    #   nn.Conv1d(64, 128, kernel_size= 10, stride = 1, padding = 2),
    #   nn.LeakyReLU(),
    #   nn.MaxPool1d(kernel_size=2, stride=6))
    self.layer3 = nn.Sequential(
      nn.Conv1d(32, 32, kernel_size= 7, stride = 2, padding = 0),
      nn.BatchNorm1d(32),
      nn.LeakyReLU())
    # self.layer4 = nn.Sequential(
    #   nn.Conv1d(128, 256, kernel_size = 5, stride = 1, padding = 2),
    #   nn.ReLU(), 
    #  nn.MaxPool1d(kernel_size=1, stride=4))
    # self.layer5 = nn.Sequential(
    #   nn.Conv1d(256, 512, kernel_size = 5, stride = 1, padding = 2),
    #   nn.ReLU(), 
    #   nn.MaxPool1d(kernel_size=1, stride=4))
    # self.layer6 = nn.Sequential(
    #   nn.Conv1d(512, 1024, kernel_size = 5, stride = 1, padding = 2),
    #   nn.ReLU(), 
    #   nn.MaxPool1d(kernel_size=1, stride=4))

    self.layer6_conv1 = nn.Sequential(
     nn.Conv1d(2, 1, kernel_size = 1, stride = 1, padding = 0))
   

    self.layer6 = nn.Sequential(nn.Dropout(p = 0.2))
    # self.fc_disease = nn.Linear(1, 1024)
    self.fc1 = nn.Linear(1024, 1)

    self.extended_tens = torch.zeros((1024, 1))
    # self.fc2 = nn.Linear(64, 1)
    # self.fc1 = nn.Sequential(nn.Linear(23328, 1))
    # self.sigmoid = nn.Sigmoid()

  def forward(self, x, hie):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    out = self.layer6(out)
    out_flatten = out.flatten(1)
    extended_hie = self.extended_tens(hie, 1024)
    out = torch.cat((out_flatten, extended_hie), dim=1)
    out = self.layer6_conv1(out)
    out = self.fc1(out)
    # out = self.sigmoid(out)
    return out


  def training_step(self, batch_loader, batch_idx):
    X, hie, Y = batch_loader
    Y_preds = self(X.float(), hie.float())
    loss = loss_fn(Y_preds, Y.float())
    acc = accuracy(Y_preds, Y, task = "binary")
    self.log('train_loss', loss) 
    self.log('train_accuracy', acc)    
    # self.log('val auc', auc)
    return {'loss': loss, 'accuracy':acc} 
    
  def validation_step(self, batch_loader, batch_idx):
    X, hie, Y = batch_loader
    Y_preds = self(X.float(), hie.float())
    loss = loss_fn(Y_preds, Y.float())
    acc = accuracy(Y_preds, Y, task = "binary")
    self.log('val_accuracy', acc)
    self.log('val_loss', loss)
    return {'loss': loss, 'accuracy':acc} 
  
  def test_step(self, batch_loader, batch_idx):
    X, hie, Y = batch_loader
    Y_preds = self(X.float(), hie.float())
    loss = loss_fn(Y_preds, Y.float())
    acc = accuracy(Y_preds, Y, task = "binary")
    self.log('test_accuracy', acc)
    self.log('test_loss', loss)
    return {'loss': loss, 'accuracy':acc} 
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-5, weight_decay=1e-5)
    return optimizer
    # scheduler = StepLR(optimizer , step_size= 100, gamma= 0.1)
    # return [optimizer], [scheduler]
    
    # weight_0 = len(train_label) / (1- sum(train_label) *2)
    # weight_1 = len(train_label) / (sum(train_label) *2)
    # weight = torch.tensor(sum(train_label)/len(train_label)).cuda()
    # loss_fn = nn.BCEWithLogitsLoss(weight = weight)
    # class_weights = compute_class_weight('balanced', np.unique(train_label), train_label)
    # class_weights_dict = dict(enumerate(class_weights))
    # loss_fn = nn.BCELoss()
    # loss_fn = nn.BCELoss(weight=weight)



In [100]:
#Resnet
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_sizes):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_sizes[0], padding=(kernel_sizes[0] // 2))
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_sizes[1], padding=(kernel_sizes[1] // 2))
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.conv3 = nn.Conv1d(out_channels, out_channels, kernel_sizes[2], padding=(kernel_sizes[2] // 2))
        self.bn3 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.downsample_bn = nn.BatchNorm1d(out_channels)

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        # out += identity
        out = self.relu(out)
        return out

class ResNet(pl.LightningModule):
    def __init__(self):
        super(ResNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=10, stride=1, padding=2),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=2, stride=6)
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 64, [10, 10, 10]),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=2, stride=6)
        )
        self.layer3 = nn.Sequential(
            ResidualBlock(64, 128, [10, 10, 10]),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=2, stride=6)
        )
        self.layer4 = nn.Sequential(
            ResidualBlock(128, 256, [5, 5, 5]),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=1, stride=4)
        )
        self.layer5 = nn.Sequential(
            ResidualBlock(256, 512, [5, 5, 5]),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=1, stride=4)
        )
        self.layer6 = nn.Sequential(nn.Dropout(p=0.9))
        self.fc_disease = nn.Linear(1, 1024)
        self.fc1 = nn.Linear(2048, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, hie):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        degree = self.fc_disease(hie)
        out = out.flatten(1)
        out = torch.cat((out, degree), dim=1)
        out = self.fc1(out)
        out = self.sigmoid(out)
        return out

    def training_step(self, batch_loader, batch_idx):
        X, hie, Y = batch_loader
        Y_preds = self(X.float(), hie.float())
        loss = nn.BCELoss(Y_preds, Y.float())
        acc = accuracy(Y_preds, Y, task = "binary")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return {'loss': loss, 'ACC': acc}

    def validation_step(self, batch_loader, batch_idx):
        X, hie, Y = batch_loader
        Y_preds = self(X.float(), hie.float())
        loss = nn.BCELoss(Y_preds, Y.float())
        acc = accuracy(Y_preds, Y, task = "binary")
        self.log('val_loss', loss)
        self.log('val_acc', acc)
        return {'loss': loss, 'ACC': acc}

    def test_step(self, batch_loader, batch_idx):
        X, hie, Y = batch_loader
        Y_preds = self(X.float(), hie.float())
        loss = loss_fn(Y_preds, Y.float())
        acc = accuracy(Y_preds, Y, task = "binary")
        self.log('test_accuracy', acc)
        self.log('test_loss', loss)
        return {'loss': loss, 'accuracy':acc} 

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-5, weight_decay=1e-5)
        return optimizer

    weight_0 = len(train_label) / (1- sum(train_label) *2)
    weight_1 = len(train_label) / (sum(train_label) *2)
    # weight = torch.tensor(sum(train_label)/len(train_label)).cuda()
    # loss_fn = nn.BCEWithLogitsLoss(weight = weight)
    # loss_fn = nn.BCELoss(weight=torch.tensor([weight_0, weight_1]))
    weight = torch.tensor([[weight_0], [weight_1]])
    loss_fn = nn.BCELoss(weight=weight)


In [56]:
#to have k-fold cross validation: 

for i in range(2):
    df = pd.read_csv('nirs with MRI outcome for CNN_10hours_overfree.csv', header = None)
    label = pd.read_csv('MRI outcome for CNN_10hours_overfree.csv', header = None)
    hie = pd.read_csv('HIE_fixed_for_MRI_overfree.csv', header = None)
    train_signal, train_label, train_hie, test_signal, test_label, test_hie, val_signal, val_label, val_hie = making_train_test_val_signals(df, label, hie, 8)
    dataset = list(zip(train_signal, train_hie, train_label))
    val_dataset = list(zip(val_signal, val_hie, val_label))
    test_dataset = list(zip(test_signal, test_hie,  test_label))
    #augmentation for training dataset
    new_dataset = generate_new_signals_from_one(dataset, 5)
    new_signal = np.array([signal for signal, _,_ in new_dataset])
    new_signal = new_signal.reshape(len(new_dataset), -1)[ :, :5890]
    new_signal = pd.DataFrame(new_signal)
    new_signal = new_signal.values
    new_signal = new_signal.reshape(-1,1,5890)
    new_hie = np.array([hie for _, hie, _ in new_dataset])
    new_hie = pd.DataFrame(new_hie)
    new_hie = new_hie.values
    new_hie = new_hie.reshape(-1,1)
    new_label = np.array([label for _, _,label in new_dataset])
    new_label = pd.DataFrame(new_label)
    new_label = new_label.values
    weight = (len(new_label) - sum(new_label))/ sum(new_label)
    # weight = 20
    print(weight)
    new_label = new_label.reshape(-1,1)
    newer_dataset = list(zip(new_signal, new_hie, new_label))
    #augmentation for validation dataset
    new_validation = generate_new_signals_from_one(val_dataset, 5)
    new_signal_val = np.array([signal for signal, _,_ in new_validation])
    new_signal_val = new_signal_val.reshape(len(new_validation), -1)[ :, :5890]
    new_signal_val = pd.DataFrame(new_signal_val)
    new_signal_val = new_signal_val.values
    new_signal_val = new_signal_val.reshape(-1,1,5890)
    new_hie_val = np.array([hie for _, hie, _ in new_validation])
    new_hie_val = pd.DataFrame(new_hie_val)
    new_hie_val = new_hie_val.values
    new_hie_val = new_hie_val.reshape(-1,1)
    new_label_val = np.array([label for _, _,label in new_validation])
    new_label_val = pd.DataFrame(new_label_val)
    new_label_val = new_label_val.values
    new_label_val = new_label_val.reshape(-1,1)
    newer_validation = list(zip(new_signal_val, new_hie_val, new_label_val))
    #augmentation for test dataset
    new_test = generate_new_signals_from_one(test_dataset, 5)
    new_signal_test = np.array([signal for signal, _,_ in new_test])
    new_signal_test = new_signal_test.reshape(len(new_test), -1)[ :, :5890]
    new_signal_test = pd.DataFrame(new_signal_test)
    new_signal_test = new_signal_test.values
    new_signal_test = new_signal_test.reshape(-1,1,5890)
    new_hie_test = np.array([hie for _, hie, _ in new_test])
    new_hie_test = pd.DataFrame(new_hie_test)
    new_hie_test = new_hie_test.values
    new_hie_test = new_hie_test.reshape(-1,1)
    new_label_test = np.array([label for _, _,label in new_test])
    new_label_test = pd.DataFrame(new_label_test)
    new_label_test = new_label_test.values
    new_label_test = new_label_test.reshape(-1,1)
    newer_test = list(zip(new_signal_test, new_hie_test, new_label_test))
    # new_hie.to_csv('fromone.csv', header=None, index=False)
    # training_dataset = pd.read_csv('fromone.csv', header = None)
    # for i in range(10):
    #     if i <4:
    #         num = 1
    #         filename = f'combined_signals_train_label_1_{i}.csv'
    #         df_combined_signals = generate_new_signals(dataset,100, 1000, 589, 1)
    #         df_combined_signals = pd.DataFrame(df_combined_signals)
    #         df_combined_signals.to_csv(filename, header=None, index=False)
    #     else:
    #         num = 0
    #         filename = f'combined_signals_train_label_0_{i-4}.csv'
    #         df_combined_signals = generate_new_signals(dataset,100, 1000, 589, 0)
    #         df_combined_signals = pd.DataFrame(df_combined_signals)
    #         df_combined_signals.to_csv(filename, header=None, index=False)

    # df_00 = pd.read_csv('combined_signals_train_label_0_0.csv', header = None)
    # df_01 = pd.read_csv('combined_signals_train_label_0_1.csv', header = None)
    # df_02 = pd.read_csv('combined_signals_train_label_0_2.csv', header = None)
    # df_03 = pd.read_csv('combined_signals_train_label_0_3.csv', header = None)
    # df_04 = pd.read_csv('combined_signals_train_label_0_4.csv', header = None)
    # df_05 = pd.read_csv('combined_signals_train_label_0_5.csv', header = None)
    # df_10 = pd.read_csv('combined_signals_train_label_1_0.csv', header = None)
    # df_11 = pd.read_csv('combined_signals_train_label_1_1.csv', header = None)
    # df_12 = pd.read_csv('combined_signals_train_label_1_2.csv', header = None)
    # df_13 = pd.read_csv('combined_signals_train_label_1_3.csv', header = None)
    # combined_signals = np.concatenate([train_signal, df_00, df_01, df_02, df_03,
    #                                df_04, df_05, df_10, df_11, df_12, df_13], axis=0)
    # arr0 = np.zeros((6000,1))
    # arr1 = np.ones((4000,1))
    # train_label = np.array(train_label)
    # train_label = train_label.reshape(-1,1)
    # combined_label = np.concatenate([train_label, arr0, arr1], axis = 0)
    # combined_label = combined_label.reshape(-1,1)
    # train_hie = np.array(train_hie)
    # train_hie = train_hie.reshape(-1,1)
    # combined_hie = np.concatenate([train_hie, arr0, arr1], axis= 0)
    # combined_hie = combined_hie.reshape(-1,1)
    # combined_signals = combined_signals.reshape(-1,1,5890)
    # combined_signals = torch.tensor(combined_signals)
    # combined_label = torch.tensor(combined_label)
    # combined_hie = torch.tensor(combined_hie)
    # dataset = list(zip(combined_signals, combined_hie, combined_label))
    
    positive_weight = torch.ones([1])* weight
    loss_fn = nn.BCEWithLogitsLoss(pos_weight = positive_weight.cuda())
    # loss_fn = torch.nn.BCELoss(weight=torch.tensor([10000.,1.]), reduction='sum')
    train_loader = DataLoader(newer_dataset, batch_size= 256, shuffle=True)
    val_loader = DataLoader(newer_validation, batch_size= 64)
    test_loader = DataLoader(newer_test)
    model = CNN()
    trainer = pl.Trainer(accelerator="gpu", devices= 1, max_epochs= 1000)
    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, test_loader)


43    47
27    29
50    55
17    19
dtype: int64
54    59
47    51
52    57
21    23
dtype: int64
20


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name       | Type       | Params
------------------------------------------
0 | layer1     | Sequential | 704   
1 | layer2     | Sequential | 41.0 K
2 | layer3     | Sequential | 82.0 K
3 | layer4     | Sequential | 164 K 
4 | layer5     | Sequential | 655 K 
5 | layer6     | Sequential | 0     
6 | fc_disease | Linear     | 2.0 K 
7 | fc1        | Linear     | 2.0 K 
------------------------------------------
947 K     Trainable params
0         Non-trainable params
947 K     Total params
3.791     Total estimated model params size (MB)


Epoch 419:  58%|█████▊    | 80/137 [00:02<00:01, 37.67it/s, loss=0.0264, v_num=190] 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing DataLoader 0: 100%|██████████| 1560/1560 [00:05<00:00, 290.17it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.28205129504203796
        test_loss            149.4357147216797
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
3      4
9     10
55    60
19    21
dtype: int64
49    54
35    38
25    27
13    15
dtype: int64
20


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name       | Type       | Params
------------------------------------------
0 | layer1     | Sequential | 704   
1 | layer2     | Sequential | 41.0 K
2 | layer3     | Sequential | 82.0 K
3 | layer4     | Sequential | 164 K 
4 | layer5     | Sequential | 655 K 
5 | layer6     | Sequential | 0     
6 | fc_disease | Linear     | 2.0 K 
7 | fc1        | Linear     | 2.0 K 
------------------------------------------
947 K     Trainable params
0         Non-trainable params
947 K     Total params
3.791     Total estimated model params size (MB)


Epoch 2:   0%|          | 0/142 [00:00<?, ?it/s, loss=2.29, v_num=191]          

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing DataLoader 0: 100%|██████████| 3120/3120 [00:11<00:00, 264.92it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.3461538553237915
        test_loss            2.199209213256836
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [22]:
print(new_label.shape)

(28800, 1)


In [18]:
#to see if the vsc is using cpu or gpu 
import torch
print(torch.cuda.is_available())

True


In [57]:
model.eval()
y_hat = model(torch.tensor(new_signal_test).float(), torch.tensor(new_hie_test).float())
auc = torch.tensor(roc_auc_score(new_label_test, y_hat.detach()))
print(auc)
prediction = (y_hat >= 0.5).int()
prediction = pd.DataFrame(y_hat)
prediction.to_csv("prediciton.csv", header=None, index=False)


tensor(0.6135, dtype=torch.float64)


In [48]:
from pytorch_lightning.loggers import tensorboard
%load_ext tensorboard
%tensorboard --logdir lightning_logs/
#for seeing the graphs, first cnt+shift+p; then search for "Python: Launch TensorBoard" 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 11528), started 10 days, 1:05:08 ago. (Use '!kill 11528' to kill it.)